## Logistic Regression: Fit and evaluate a model

Using the Titanic dataset from [this](https://www.kaggle.com/c/titanic/overview) Kaggle competition.

In this section, we will fit and evaluate a simple Logistic Regression model.

### Read in Data

<!-- ![CV](img/lr/CV.png)
![Cross-Val](img/lr/Cross-Val.png) -->
<table><tr><td><img src="img/lr/CV.png" width="1000"></td><td><img src="img/lr/Cross-Val.png" width="1000"></td></tr></table>

In [8]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

feature_columns = ['Pclass', 'Gender', 'Age', 'Fare', 'Family_count', 'Cabin_index']
label_column = ['Survival']

tr_features = pd.read_csv('dataset/train_dataset.csv', usecols=feature_columns)
tr_labels = pd.read_csv('dataset/train_dataset.csv', usecols=label_column)

### Hyperparameter tuning

<img src="img/lr/c.png" width="500">

In [9]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [10]:
lr = LogisticRegression(max_iter=1000) # max_iter=1000
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# Grid search with 5 fold cross validation
cv = GridSearchCV(lr, parameters, cv=5)
cv.fit(tr_features, tr_labels.values.ravel())

print_results(cv)

BEST PARAMS: {'C': 1}

0.67 (+/-0.077) for {'C': 0.001}
0.708 (+/-0.098) for {'C': 0.01}
0.777 (+/-0.134) for {'C': 0.1}
0.8 (+/-0.118) for {'C': 1}
0.794 (+/-0.116) for {'C': 10}
0.794 (+/-0.116) for {'C': 100}
0.794 (+/-0.116) for {'C': 1000}


In [11]:
cv.best_estimator_

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Write out pickled model

In [12]:
joblib.dump(cv.best_estimator_, 'models/lr_model.pkl')

['models/lr_model.pkl']

### Load the Logistic Regression model

In [13]:
#Loading the saved model
pipe = joblib.load('models/lr_model.pkl')

### Predict

In [14]:
# test data to predict
pr = pd.read_csv('dataset/test_dataset.csv')
# remove the label column
pred_cols = list(pr.columns.values)[:-1]
# apply the whole pipeline to data
pred = pd.Series(pipe.predict(pr[pred_cols]))

# set of predictions
print("Prediction")
print(pred[20:35].tolist())

# select label column
label_cols = list(pr.columns.values)[-1]

# corresponding actual labels
print("\nActual labels")
print(pr[label_cols][20:35].tolist())

Prediction
[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0]

Actual labels
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
